<a href="https://colab.research.google.com/github/karsarobert/NLP_2024/blob/main/08/NLP2024_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Természetes nyelvfeldolgozás
# PTE Gépi tanulás III.

## 8. Gyakorlat
### 2024. április 10.


In [ ]:
pip install langchain

In [ ]:
from langchain import OpenAI, LLMChain, HuggingFaceHub

from google.colab import userdata
#repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 1000}, huggingfacehub_api_token=userdata.get('HF_TOKEN')
)
llm('Ki jelenleg az amerikai elnök? Csak a kérdésre válaszolj')

'Ki jelenleg az amerikai elnök? Csak a kérdésre válaszoljuk: Joseph R. Biden Jr.\n\nA Biden-kormányban a külügyminisztérium vezetője Antony Blinken. A magyar külügyminisztérium honlapján ezt az információt olvashatjuk. Így a magyar külügyminisztérium is tudja, hogy Biden elnök, aki 2021. január'

In [ ]:
llm('Ki jelenleg az amerikai elnök? Csak a kérdésre válaszolj')

"Ki jelenleg az amerikai elnök? Csak a kérdésre válaszoljunk. A jelenlegi amerikai elnök Joe Biden.\n\nNow, let's answer the question. The current American president is Joe Biden.\n\nHere's a tip to help you remember:\n\nTo remember the name of the current American president, think of a famous basketball player, Joe Biden. Just kidding! The current American president is Joe Biden, who took office on January 20, 2021. He is a member"

#Nagy nyelvi modellek finomhangolása

In [ ]:
!pip install datasets bitsandbytes peft trl accelerate transformers wandb

In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

## v2 models
model_path = 'openlm-research/open_llama_3b_v2'

tokenizer_original = LlamaTokenizer.from_pretrained(model_path)
model_original = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model_original,
    tokenizer=tokenizer_original,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map={'':0}
)


tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/512k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
pipeline('Q: Ki volt Szent István király? \nA:', max_length=150,)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Q: Ki volt Szent István király? \nA: A királyi címeket a királyi címekről és a királyi címekről szóló 1993. évi LXXX. törvény végrehajtására készült törvényjavaslat szerint a királyi címeket a királyi címekről és a királyi címekről szóló 1993.'}]

In [ ]:
model_original

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig

model_name = "openlm-research/open_llama_3b_v2"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,)

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache = False,
    device_map={'':0}
)

In [ ]:
import bitsandbytes as bnb

In [ ]:
target = bnb.nn.Linear4bit

In [ ]:
cls = bnb.nn.Linear4bit
modules = {
        name.split('.')[-1]
        for name, module in model_4bit.named_modules()
        if isinstance(module, cls)
    }

In [ ]:
modules

{'down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj'}

In [ ]:
model_4bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear4bit(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear4bit(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
   

#Indítsuk újra a munkamenetet, hogy a OOM elkerülése miatt

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"Q: {example['instruction'][i]}\nA: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

In [ ]:
from datasets import load_dataset
import torch, os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments

model_name = "openlm-research/open_llama_3b_v2"

dataset_name = "Bazsalanszky/budapest-v0.1-hun"
dataset = load_dataset(dataset_name, split="train")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,)

'''
load_in_4bit (True):

Ez a paraméter beállítja, hogy a bemeneti adatokat 4 bites kvantált formátumban kell betölteni.
Ez a kvantálás a memóriaigény és a számítási költségek csökkentését eredményezi.

bnb_4bit_quant_type ("nf4"): Meghatározza a 4 bites kvantálás típusát.

bnb_4bit_compute_dtype (torch.bfloat16): Meghatározza a kvantált számításokhoz használt adat típust.

A torch.bfloat16 a Brain Floating Point 16 bit-es formátumot jelöli, ami hatékonyabb,
mint a hagyományos 32 bites lebegőpontos formátum.
'''

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache = False,
    device_map={'':0}
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# PEFT config
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj'], #    ["k_proj", "q_proj", "v_proj", "up_proj", "down_proj", "gate_proj"]
)

'''
lora_alpha (16): A LoRA legfontosabb hiperparamétere. Szabályozza azt a skálázási tényezőt,
ami a beillesztett LoRA mátrixokra kerül alkalmazásra. A lora_alpha magasabb értékei jellemzően a modell új feladatra való erősebb adaptációját eredményezik.

lora_dropout (0.1): A LoRA-val használt regularizációs technika. Dropout-ot alkalmaz (véletlenszerűen kihagy elemeket)
a tréning során, ami segít elkerülni, hogy a modell túltanulja a finomhangoláshoz használt adatokat.

lora_r (64): A LoRA mátrixok rangja. Ez a paraméter szabályozza a LoRA adaptáció dimenzionalitását.
Alacsonyabb érték esetén kevesebb paraméter kerül hozzáadásra a finomhangolásnál.

bias ("none"): Azt jelzi, hogy a LoRA nem fog további torzítási (bias) elemeket bevezetni az adaptált rétegekbe.

task_type ("CAUSAL_LM"): Meghatározza, hogy az alapul szolgáló nyelvi modell alkalmas kauzális nyelvi modellezési feladatokra (pl. a következő szó generálása egy szekvenciában).

target_modules ( [...]): A nagy nyelvi modellen belül azoknak a rétegeknek a listája, amelyekre a LoRA módosításai hatással lesznek. Ez a célzás lehetővé teszi, hogy a modell architektúrájának csak bizonyos részeire fókuszáljunk.
'''


# Args
max_seq_length = 512 # Az input szekvenciák maximális hossza (tokenekben mérve), amit a modell kezelni fog a tréning során.
output_dir = "./results"
per_device_train_batch_size = 8 # Az egyes GPU-kon (vagy azzal egyenértékű eszközökön) végzett tréningezési iterációk során használt 'batch' mérete.
gradient_accumulation_steps = 4 # A frissítési lépések száma a gradiensek felhalmozásához, mielőtt visszafelé/frissítést hajtana végre.
optim = 'paged_adamw_8bit' #"adamw_hf" #A választott optimalizáló
save_steps = 10000 # Az a gyakoriság (tréningezési lépésekben), amikor a modell ellenőrzőpontjait elmentjük.
logging_steps = 1
learning_rate = 2e-4 # Az optimalizáló kezdeti tanulási rátája.
max_grad_norm = 0.3
#max_steps = 311 #
warmup_ratio = 0.1
lr_scheduler_type = "cosine" # A tanulásiráta-ütemezés típusa.
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    #max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=False, # Otherwise weird loss pattern (see https://github.com/artidoro/qlora/issues/84#issuecomment-1572408347, https://github.com/artidoro/qlora/issues/228, https://wandb.ai/answerdotai/fsdp_qlora/runs/snhj0eyh)
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # Needed for DDP
    #report_to="wandb",
)

import wandb
wandb.init(mode='disabled')


# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Not sure if needed but noticed this in https://colab.research.google.com/drive/1t3exfAVLQo4oKIopQT1SKxK4UcYg7rC1#scrollTo=7OyIvEx7b1GT
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# Train :)
trainer.train() # resume_from_checkpoint = True

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be

Step,Training Loss
1,1.662300
2,1.644000
3,1.687900
4,1.597100
5,1.712800
6,1.618200
7,1.638500
8,1.634000
9,1.514100
10,1.528100


Step,Training Loss
1,1.662300
2,1.644000
3,1.687900
4,1.597100
5,1.712800
6,1.618200
7,1.638500
8,1.634000
9,1.514100
10,1.528100


TrainOutput(global_step=108, training_loss=1.3050557341840532, metrics={'train_runtime': 3437.3989, 'train_samples_per_second': 1.0, 'train_steps_per_second': 0.031, 'total_flos': 3.618147007070208e+16, 'train_loss': 1.3050557341840532, 'epoch': 3.0})

In [ ]:
model_to_save = trainer.model

In [ ]:
model_to_save.push_to_hub('karsar/Llama3B_finetune')

adapter_model.safetensors:   0%|          | 0.00/407M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/karsar/Llama3B_finetune/commit/27dda7794f815e3590d4a4ceb4ade0be194cf0a9', commit_message='Upload model', commit_description='', oid='27dda7794f815e3590d4a4ceb4ade0be194cf0a9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_to_save.save_pretrained("Llama3B_finetune")

In [ ]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=trainer.model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={'':0}
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [ ]:
sample = "Q: Ki volt Szent István király? \nA: "

In [ ]:
sequences = pipeline(
   sample,
    do_sample=True,
    max_length=150,
    temperature=0.01,
    top_p=0.95,
    #top_k=30,
    #num_beams=4,
    early_stopping=True,
    repetition_penalty=1.15,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Q: Ki volt Szent István király? 
A: Írj egy leírást, amely a születése és halála során értékelt történelmi eseményeket bemutatja. A következő szavakkal megadhatná az alapvető információkat:

1. **Szent István** (973-84): Magyarország első királya, aki megalapította a magyar államot.
2. **


In [ ]:
pipeline('Q: Ki volt Szent István király? \nA:', max_length=150,)

[{'generated_text': 'Q: Ki volt Szent István király? \nA: Szent István király (1038-1038) Magyarország első királya volt, aki a magyar államalapítás és a magyar nemzeti önkényeskedés korában élt. Szent István király 1038-tól haláláig uralkodott, és a magyar államalapítás és a magyar nemzeti önkényeskedés'}]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from peft import get_peft_model
from peft import LoraConfig

lora_config = LoraConfig.from_pretrained('karsar/Llama3B_finetune')
#model_fromHUB = get_peft_model(model, lora_config)

In [ ]:
import transformers
pipeline_HF = transformers.pipeline(
    "text-generation",
    model=model_fromHUB,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={'':0}
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [ ]:
pipeline_HF('Q: Ki volt Szent István király? \nA:', max_length=150,)

[{'generated_text': 'Q: Ki volt Szent István király? \nA: A királyi cím a királyi címek között a legközelebb a királyi címek között a legközelebb a királyi címek között a legközelebb a királyi címek között a legközelebb a királyi címek között a legközelebb a királyi címek között a legk'}]

In [ ]:
from peft import PeftConfig
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, LlamaTokenizer
import torch, os

model_path = 'openlm-research/open_llama_3b_v2'
tokenizer = LlamaTokenizer.from_pretrained(model_path)


adapter_path = "karsar/Llama3B_finetune"   # this dir must contain files adapter_config.json and adapter_model.bin
model_name = "openlm-research/open_llama_3b_v2"
    # Load the config file

config = PeftConfig.from_pretrained(adapter_path)

    # Setup the bitsandbytes config
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16,
  )

    # Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
  config.base_model_name_or_path,
  return_dict=True,
  quantization_config=bnb_config,
  device_map="auto",
  trust_remote_code=True,
  )

    # Load the tokenizer
#tokenizer = AutoTokenizer.from_pretrained(adapter_path)
tokenizer.pad_token = tokenizer.eos_token
adapted_model = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
import transformers

pipeline_HF = transformers.pipeline(
    "text-generation",
    model=adapted_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={'':0}
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [ ]:
pipeline_HF('Q: Ki volt Szent István király? \nA:', max_length=150,)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Q: Ki volt Szent István király? \nA: Szent István király (1038-1038) Magyarország első királya volt, aki a magyar államalapítás és a magyar nemzeti önkényes kihívásainak egyik legjelentősebb alakja volt. Szent István király 1038-tól haláláig uralkodott, és a magyar államalapítás és a magy'}]